# Module 6 - Topic Modelling - Exercises

1. Run the module notebook yourself without referring to the video
2. Make a different selection from the science stories used in the original notebook, and change the variables appropriately
3. Perform an LDA from scratch using your subject selection, and experiment with different values of K

In [1]:
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from gensim.utils import tokenize
from gensim.utils import simple_preprocess
from gensim.corpora.textcorpus import remove_stopwords
from gensim.summarization import keywords
from gensim.models.ldamodel import LdaModel
import pyLDAvis
import pyLDAvis.gensim

from pattern.en import lemma

import pandas as pd
import json

In [2]:
# load the complete dataset
with open('data/News_Category_Dataset_v2.json', 'r') as f:
    news_list = f.readlines()

# convert each line (string) to json (dict)
news_json = list(map(json.loads,news_list))

print("Number of stories: ",len(news_json))

# view the first 10 elements in the list
news_json[:20]

C:\Users\mrevi\anaconda3\envs\qut\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Number of stories:  200853


[{'category': 'CRIME',
  'headline': 'There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV',
  'authors': 'Melissa Jeltsen',
  'link': 'https://www.huffingtonpost.com/entry/texas-amanda-painter-mass-shooting_us_5b081ab4e4b0802d69caad89',
  'short_description': 'She left her husband. He killed their children. Just another day in America.',
  'date': '2018-05-26'},
 {'category': 'ENTERTAINMENT',
  'headline': "Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song",
  'authors': 'Andy McDonald',
  'link': 'https://www.huffingtonpost.com/entry/will-smith-joins-diplo-and-nicky-jam-for-the-official-2018-world-cup-song_us_5b09726fe4b0fdb2aa541201',
  'short_description': 'Of course it has a song.',
  'date': '2018-05-26'},
 {'category': 'ENTERTAINMENT',
  'headline': 'Hugh Grant Marries For The First Time At Age 57',
  'authors': 'Ron Dicker',
  'link': 'https://www.huffingtonpost.com/entry/hugh-grant-marries_us_5b09212ce4b0568a880b9a8c',
  'short_description

In [3]:
# filter the list for stories that are in the category SCIENCE
science_json = [story for story in news_json if story['category']=='SCIENCE']

# for each, create the 'story' by adding together the headline and the short_description
science_stories = [story['headline']+' - '+story['short_description'] for story in science_json]

print("Number of science stories: ",len(science_stories))

Number of science stories:  2178


C:\Users\mrevi\anaconda3\envs\qut\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
science_stories

C:\Users\mrevi\anaconda3\envs\qut\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['Scientists Turn To DNA Technology To Search For Loch Ness Monster - The researchers plan to scour the Loch Ness next month for evidence of its supposed inhabitant.',
 'Unusual Asteroid Could Be An Interstellar Guest To Our Solar System - The supposed "interstellar immigrant" is located near Jupiter but has an atypical orbit.',
 "China Marks Another Milestone In Quest To Become Space Superpower - It's the first time a rocket designed by a Chinese private company has successfully entered orbit.",
 'Terrifying Clip Shows Why You Should Never Run Under A Tree During Thunderstorms - YIKES!',
 "U.S. Climate Scientists Flee For France To ‘Make Our Planet Great Again’ - Some of America's top researchers will move to France to continue their research.",
 'Stephen Hawking Finished Mind-Bending Parallel Universe Paper Days Before His Death - The new treatise on the existence of parallel universes was published on Friday.',
 "Mysterious Yellowstone Geyser Eruptions Stump Scientists - Steamboat b

In [5]:
# Stories to DF
stories_df = pd.DataFrame(science_stories,columns=['story'])
stories_df

C:\Users\mrevi\anaconda3\envs\qut\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,story
0,Scientists Turn To DNA Technology To Search Fo...
1,Unusual Asteroid Could Be An Interstellar Gues...
2,China Marks Another Milestone In Quest To Beco...
3,Terrifying Clip Shows Why You Should Never Run...
4,U.S. Climate Scientists Flee For France To ‘Ma...
...,...
2173,Treating a World Without Antibiotics? - Becaus...
2174,Russian Cargo Ship Docks At International Spac...
2175,"Robots Play Catch, Starring Agile Justin And R..."
2176,Thomas Edison Voted Most Iconic Inventor In U....


In [25]:
# do this for whole dataframe
stories_df['terms'] = [remove_stopwords(simple_preprocess(story,min_len=3)) for story in stories_df['story']]
stories_df

C:\Users\mrevi\anaconda3\envs\qut\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,story,terms
0,Scientists Turn To DNA Technology To Search Fo...,"[scientists, turn, dna, technology, search, lo..."
1,Unusual Asteroid Could Be An Interstellar Gues...,"[unusual, asteroid, interstellar, guest, solar..."
2,China Marks Another Milestone In Quest To Beco...,"[china, marks, milestone, quest, space, superp..."
3,Terrifying Clip Shows Why You Should Never Run...,"[terrifying, clip, shows, run, tree, thunderst..."
4,U.S. Climate Scientists Flee For France To ‘Ma...,"[climate, scientists, flee, france, planet, gr..."
...,...,...
2173,Treating a World Without Antibiotics? - Becaus...,"[treating, world, antibiotics, overuse, antibi..."
2174,Russian Cargo Ship Docks At International Spac...,"[russian, cargo, ship, docks, international, s..."
2175,"Robots Play Catch, Starring Agile Justin And R...","[robots, play, catch, starring, agile, justin,..."
2176,Thomas Edison Voted Most Iconic Inventor In U....,"[thomas, edison, voted, iconic, inventor, hist..."


In [27]:
stories_df['lemma'] = [[lemma(wd) for wd in terms] for terms in stories_df['terms']]
stories_df

C:\Users\mrevi\anaconda3\envs\qut\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,story,terms,lemma
0,Scientists Turn To DNA Technology To Search Fo...,"[scientists, turn, dna, technology, search, lo...","[scientist, turn, dna, technology, search, loc..."
1,Unusual Asteroid Could Be An Interstellar Gues...,"[unusual, asteroid, interstellar, guest, solar...","[unusual, asteroid, interstellar, guest, solar..."
2,China Marks Another Milestone In Quest To Beco...,"[china, marks, milestone, quest, space, superp...","[china, mark, milestone, quest, space, superpo..."
3,Terrifying Clip Shows Why You Should Never Run...,"[terrifying, clip, shows, run, tree, thunderst...","[terrify, clip, show, run, tree, thunderstorm,..."
4,U.S. Climate Scientists Flee For France To ‘Ma...,"[climate, scientists, flee, france, planet, gr...","[climate, scientist, flee, france, planet, gre..."
...,...,...,...
2173,Treating a World Without Antibiotics? - Becaus...,"[treating, world, antibiotics, overuse, antibi...","[treat, world, antibiotic, overuse, antibiotic..."
2174,Russian Cargo Ship Docks At International Spac...,"[russian, cargo, ship, docks, international, s...","[russian, cargo, ship, dock, international, sp..."
2175,"Robots Play Catch, Starring Agile Justin And R...","[robots, play, catch, starring, agile, justin,...","[robot, play, catch, star, agile, justin, roll..."
2176,Thomas Edison Voted Most Iconic Inventor In U....,"[thomas, edison, voted, iconic, inventor, hist...","[thoma, edison, vote, iconic, inventor, histor..."


In [28]:
vocab = Dictionary(stories_df['lemma'])
print(vocab.token2id)

{'dna': 0, 'evidence': 1, 'inhabitant': 2, 'loch': 3, 'monster': 4, 'month': 5, 'nes': 6, 'plan': 7, 'researcher': 8, 'scientist': 9, 'scour': 10, 'search': 11, 'suppose': 12, 'technology': 13, 'turn': 14, 'asteroid': 15, 'atypical': 16, 'guest': 17, 'immigrant': 18, 'interstellar': 19, 'jupiter': 20, 'locate': 21, 'near': 22, 'orbit': 23, 'solar': 24, 'unusual': 25, 'china': 26, 'chinese': 27, 'company': 28, 'design': 29, 'enter': 30, 'mark': 31, 'milestone': 32, 'private': 33, 'quest': 34, 'rocket': 35, 'space': 36, 'successfully': 37, 'superpower': 38, 'time': 39, 'clip': 40, 'run': 41, 'show': 42, 'terrify': 43, 'thunderstorm': 44, 'tree': 45, 'yike': 46, 'america': 47, 'climate': 48, 'continue': 49, 'flee': 50, 'france': 51, 'great': 52, 'planet': 53, 'research': 54, 'bend': 55, 'day': 56, 'death': 57, 'existence': 58, 'finish': 59, 'friday': 60, 'hawk': 61, 'mind': 62, 'new': 63, 'paper': 64, 'parallel': 65, 'publish': 66, 'stephen': 67, 'treatise': 68, 'universe': 69, 'blow': 70

C:\Users\mrevi\anaconda3\envs\qut\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
# convert corpus to BoW format
corpus = [vocab.doc2bow(terms) for terms in stories_df['lemma']]

C:\Users\mrevi\anaconda3\envs\qut\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [39]:
# create an lda model from our corpus and vocab - we need to specify the number of topics
lda_model = LdaModel(corpus=corpus, id2word=vocab, num_topics=4)

C:\Users\mrevi\anaconda3\envs\qut\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [40]:
# view the topics in the model
for topic in lda_model.show_topics(num_topics=4,num_words=15):
    print("Topic "+str(topic[0])+"\n"+topic[1]+"\n")

Topic 0
0.014*"space" + 0.011*"video" + 0.009*"nasa" + 0.008*"photo" + 0.007*"earth" + 0.007*"new" + 0.006*"planet" + 0.006*"science" + 0.006*"scientist" + 0.005*"study" + 0.005*"like" + 0.004*"way" + 0.004*"asteroid" + 0.004*"light" + 0.004*"moon"

Topic 1
0.011*"study" + 0.010*"space" + 0.010*"planet" + 0.009*"scientist" + 0.008*"show" + 0.008*"new" + 0.007*"science" + 0.007*"nasa" + 0.006*"publish" + 0.006*"earth" + 0.006*"com" + 0.006*"follow" + 0.005*"star" + 0.005*"edt" + 0.005*"moon"

Topic 2
0.016*"video" + 0.011*"new" + 0.008*"study" + 0.007*"scientist" + 0.007*"show" + 0.006*"time" + 0.006*"robot" + 0.005*"research" + 0.005*"venu" + 0.005*"science" + 0.005*"higg" + 0.005*"human" + 0.005*"space" + 0.005*"aurora" + 0.005*"world"

Topic 3
0.012*"space" + 0.009*"say" + 0.008*"science" + 0.006*"day" + 0.006*"scientist" + 0.005*"year" + 0.005*"photo" + 0.005*"shuttle" + 0.004*"nasa" + 0.004*"life" + 0.004*"mar" + 0.004*"little" + 0.003*"fool" + 0.003*"astronaut" + 0.003*"world"



C:\Users\mrevi\anaconda3\envs\qut\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [41]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, vocab)
vis

C:\Users\mrevi\anaconda3\envs\qut\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.022889 -0.008169       1        1  29.133936
1      0.031420 -0.111621       2        1  25.034414
2     -0.111414  0.042017       3        1  24.895682
3      0.102883  0.077773       4        1  20.935968, topic_info=         Term        Freq       Total Category  logprob  loglift
1723   aurora   41.000000   41.000000  Default  30.0000  30.0000
4551     higg   42.000000   42.000000  Default  29.0000  29.0000
127     video  290.000000  290.000000  Default  28.0000  28.0000
53     planet  157.000000  157.000000  Default  27.0000  27.0000
666     robot   56.000000   56.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
655    people   20.619953   82.602916   Topic4  -5.7985   0.1759
137     world   21.448503  106.760693   Topic4  -5.7591  -0.0412
127     video   19.735350  290.707741   Topic4  -5.8423  -1.1262
1595  ability   15.298611   33.286578   Topic4  -6.0970   0.7863
63        new   18.594217  240.569043   Topic4  -5.9019  -0.9965

[336 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1595      1  0.030042    ability
1595      2  0.360506    ability
1595      3  0.180253    ability
1595      4  0.450632    ability
3567      2  0.103320   accurate
...     ...       ...        ...
162       1  0.296715       year
162       2  0.278171       year
162       3  0.092724       year
162       4  0.333805       year
6612      3  0.980483  zebrafish

[595 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4])